In [ ]:
# !pip install diffusers["torch"] 
# !pip install -U bitsandbytes transformers

In [ ]:
from huggingface_hub import notebook_login

#notebook_login()

# CLI
# huggingface-cli login

In [ ]:
from diffusers import BitsAndBytesConfig, SD3Transformer2DModel
from diffusers import StableDiffusion3Pipeline
import torch

model_id = "stabilityai/stable-diffusion-3.5-medium"

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_nf4 = SD3Transformer2DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=nf4_config,
    torch_dtype=torch.bfloat16
)

pipeline = StableDiffusion3Pipeline.from_pretrained(
    model_id, 
    transformer=model_nf4,
    torch_dtype=torch.bfloat16
)
pipeline.enable_model_cpu_offload()



In [ ]:

prompt = "Create a high-resolution image of a modern kitchen with wooden cabinets and stainless steel appliances, featuring a clear glass bottle on a wooden table that contains a vibrant galaxy with swirling nebulae and stars."

image = pipeline(
    prompt=prompt,
    num_inference_steps=100,
    guidance_scale=7.0,
    max_sequence_length=512,
).images[0]
image